In [1]:
import subprocess

def extract_audio(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command)


video_path = "./files/videoplayback.mp4"
audio_path = "./files/audio.mp3"

extract_audio(video_path, audio_path)

In [20]:
from pydub import AudioSegment
import math

def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    overlap_size = 10 * 1000
    chunk_len = chunk_size * 60 * 1000 - overlap_size
    chunks = math.ceil(len(track) / chunk_len)
    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len + overlap_size

        chunk = track[start_time:end_time]

        chunk.export(f"./{chunks_folder}/chunk_{i:02d}.mp3", format="mp3")

cut_audio_in_chunks(audio_path, 10, "./files/chunks")

In [ ]:
import openai
import glob


def transcript_chunks(chunks_folder, destination):
    chunks = glob.glob(f"{chunks_folder}/*.mp3")
    for chunk in chunks:
        with open(chunk, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = openai.audio.transcriptions.create(
                model="whisper-1", file=audio_file, language="en"
            )
            text_file.write(transcript.text)


transcript_chunks("./files/chunks", "./files/transcript.txt")

In [ ]:
import os
import yt_dlp


def download_audio_from_youtube(youtube_url, output_path):
    if os.path.exists(output_path):
        os.remove(output_path)

    ydl_opts = {
        "format": "bestaudio/best",
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",
                "preferredcodec": "mp3",
                "preferredquality": "192",
            }
        ],
        "outtmpl": output_path,
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

[youtube] Extracting URL: https://www.youtube.com/watch?v=fRBHJk9crCo
[youtube] fRBHJk9crCo: Downloading webpage
[youtube] fRBHJk9crCo: Downloading tv client config
[youtube] fRBHJk9crCo: Downloading player 91201489
[youtube] fRBHJk9crCo: Downloading tv player API JSON
[youtube] fRBHJk9crCo: Downloading ios player API JSON
[youtube] fRBHJk9crCo: Downloading m3u8 information
[info] fRBHJk9crCo: Downloading 1 format(s): 251
[download] Destination: files\temp_audio
[download] 100% of    6.56MiB in 00:00:00 at 8.79MiB/s   
[ExtractAudio] Destination: files\temp_audio.mp3
Deleting original file files\temp_audio (pass -k to keep)
